In [1]:
import os
import pandas as pd
import numpy as np
import glob

In [10]:
sub = 'sub-12'
ses = 'ses-01'
battery = 'camcan'
task = 'emotion-recognition'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob('*' + task + '*.csv'))
tsv_files = []
# for i in range(len(csv_files)):
i = 1
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [23]:
tsv_files

['emotion-recognition_sub-05_run-02.tsv']

In [8]:
csv_files

['extracted_emotion-recognition_sub-07_run-01_232021-152.csv',
 'extracted_emotion-recognition_sub-07_run-02_232021-1513.csv']

In [11]:
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]))

In [12]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[~np.isnan(file_df['GetReady.OffsetTime'])])['GetReady.OffsetTime']
run_start = run_start.values[0]

In [13]:
run_start

61306.0

In [14]:
# Find the trial-related rows
trials = ~np.isnan(file_df['afac1.OnsetTime']) | ~np.isnan(file_df['nfac1.OnsetTime'])
angry_trials = ~np.isnan(file_df['afac1.OnsetTime'])
neutral_trials = ~np.isnan(file_df['nfac1.OnsetTime'])

In [15]:
# Extract the gender of the displayed face
im = file_df.loc[trials]['image']
gender = ["female" if x[0]=='f' else "male" for x in im]

In [16]:
# Trial onsets are in 'afac1.OnsetTime' and 'nfac1.OnsetTime' 
# for angry and neutral trials, respectively
angry_onsets = file_df.loc[angry_trials]['afac1.OnsetTime']
neutral_onsets = file_df.loc[neutral_trials]['nfac1.OnsetTime']
trial_onsets = np.sort(np.hstack((angry_onsets, neutral_onsets)))

# Trial offsets are in 'afac1.OffsetTime' 'nfac1.OffsetTime'
# for angry and neutral trials, respectively
angry_offsets = file_df.loc[angry_trials]['afac1.OffsetTime']
neutral_offsets = file_df.loc[neutral_trials]['nfac1.OffsetTime']

# Calculate trial durations
angry_durations = angry_offsets - angry_onsets
neutral_durations = neutral_offsets - neutral_onsets
durations = np.hstack((angry_durations, neutral_durations))

# Finally, trial_types are in 'Emotion'
trial_types = (file_df.loc[trials]['Emotion']).str.lower()
trial_types = [i+'_'+j for i, j in zip(trial_types, gender)]

In [11]:
# Responses are in 'afac1.RESP' for angry trials andnfac1.RESP for neutral trials.
# RTs are in afac1.RT and fixa1.RT for angry trials, and
# nfac1.RT and fixn1.RT for neutral trials.

# These field are not used in tsv file generation but are useful 
# for behavioral analysis.

angry_responses1 = file_df.loc[angry_trials]['afac1.RESP']
# angry_responses2 = file_df.loc[angry_trials]['fixa1.RESP']
angry_RT1 = file_df.loc[angry_trials]['afac1.RT']
angry_RT2 = file_df.loc[angry_trials]['fixa1.RT']
print(np.mean(angry_RT1))

neutral_responses1 = file_df.loc[neutral_trials]['nfac1.RESP']
# neutral_responses2 = file_df.loc[neutral_trials]['fixn1.RESP']
neutral_RT1 = file_df.loc[neutral_trials]['nfac1.RT']
neutral_RT2 = file_df.loc[neutral_trials]['fixn1.RT']
print(np.mean(neutral_RT1))

621.1481481481482
612.2407407407408


In [17]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start)/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)